In [0]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-5o4c9t5_
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-5o4c9t5_
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=b79e1e79534502ae36d70d576004c60bf1667075ac604aa0c6d65a790b89b6a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-yx44whim/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [0]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <bits/stdc++.h>

#define DIMBLOCKX 32

__global__ void SumaColMatrizKernel (int M, int N, float *Md, float *Nd){
  __shared__ float Nds[DIMBLOCKX];
  float Pvalue = 0;
  int columna = threadIdx.y;//blockIdx.y * (N/gridDim.x) + threadIdx.x;
  int pasos = M/blockDim.x;
  int posIni = columna * M + threadIdx.x * pasos;

  for (int k=0; k<pasos; k++){
      Pvalue += Md[posIni + k];
  }

  atomicAdd(&Nds[threadIdx.x],Pvalue);
  
  __syncthreads();

  if (threadIdx.x == 0){
      for (int i=1; i<blockDim.x; i++){
          Nds[0] += Nds[i];
      }
      atomicAdd(&Nd[blockIdx.y],Nds[0]);
  }
  
  
}  

void SumaColMatriz (int M, int N, float *Mh, float *Nh){
    
  int size = M*N*sizeof(float), size2=N*sizeof(float);
  float *Md, *Nd;

  cudaMalloc(&Md, size);
  cudaMalloc(&Nd, size2);

  cudaMemcpy(Md, Mh, size,  cudaMemcpyHostToDevice);
  cudaMemset(Nd, 0, size2);

  int chunk = 32;
  dim3 tamGrid(N/chunk, 1);
  dim3 tamBlock(M/chunk, chunk, 1);
  SumaColMatrizKernel<<<tamGrid, tamBlock>>> (M, N, Md, Nd);

  cudaMemcpy(Nh,Nd, size2, cudaMemcpyDeviceToHost);

  cudaFree(Md); cudaFree(Nd);
}

int main(){

  int M = 1024;
  int N = 512;

  float *Mh = new float[M*N];//matriz host
  for (int i=0; i<M; i++)
    for (int j=0; j<N; j++)
      Mh[i*N+j] = 1;

  float *Nh = new float[N];//suma de columnas

  SumaColMatriz(M,N,Mh,Nh);

  for (int i=0; i<N; i++)
  std::cout<<Nh[i]<<" ";

  return 0;
}

1.8476e+07 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [0]:
%%cu
#include <iostream>

using namespace std;
// Suma por columnas de una matriz

__global__ void SumaColMatrizKernel(int M, float* Md, float* Nd){
    // Pvalue es usado para el valor intermedio
    float Pvalue = 0;
    int columna = threadIdx.x;
    int posIni = columna*M;
    for (int k = 0; k < M; ++k) {
      Pvalue = Pvalue + Md[posIni+k];
    }
    Nd[columna] = Pvalue;
  }

void SumaColMatriz(int M, int N, float * Mh, float*  Nh){
    int size = M * N * sizeof(float), size2 = N*sizeof(float);
    float* Md, *Nd;
    // Allocate en device
    cudaMalloc(&Md, size);
    cudaMalloc(&Nd, size2);
  
    // Inicializo matrices en el device
    cudaMemcpy(Md, Mh, size, cudaMemcpyHostToDevice);
    cudaMemset(Nd,0, size2);

    //SumaColMatrizKernel <<<1, N>>>(M, Md, Nd);
    
    dim3 tamGrid(1,1);
    dim3 tamBlock(N,1,1);
    SumaColMatrizKernel <<<tamGrid, tamBlock>>>(M, Md, Nd);

    // Traer resultado;
    cudaMemcpy(Nh, Nd, size2, cudaMemcpyDeviceToHost);
    for (int i=0; i<N; i++)
      std::cout<<Nh[i]<<" ";

    // Free matrices en device
    cudaFree(Md); cudaFree(Nd);
}

int main(){
    int M =1024;
    int N =512;
    float *Mh = new float[M*N];//matriz host
    float *Nh = new float[N];//matriz host
    for (int i=0; i<M; i++)
      for (int j=0; j<N; j++)
        Mh[i*N+j] = 1;

    SumaColMatriz(M,N,Mh,Nh);
    cout<<"Erika animo tú puedes de verdad"<<endl;

}

1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 

In [0]:
%%cu
#include <iostream>

using namespace std;
// Suma por columnas de una matriz
__global__ void SumaColMatrizKernel_2(int M, float* Md,float* Nd){
    // Pvalue es usado para el valor intermedio
    float Pvalue = 0;
    int columna = blockIdx.x* blockDim.x + threadIdx.x;
    int posIni = columna * M;
    for (int k = 0; k < M; ++k) {
       Pvalue = Pvalue + Md[posIni+k];
    }
    Nd[columna] = Pvalue;
}

// Suma por columnas de una matriz
__global__ void SumaColMatrizKernel_3(int M,float* Md,float* Nd){
    // Pvalue es usado para el valor intermedio
    float Pvalue = 0;
    int columna = blockIdx.x;
    int pasos = M/ blockDim.x;
    int posIni = columna * M + threadIdx.x * pasos;
    for (int k = 0; k < pasos; ++k) {
      Pvalue = Pvalue + Md[posIni + k];
    }
    atomicAdd(&(Nd[columna]), Pvalue);
}

#define DIMBLOCKX 32
__global__ void SumaColMatrizKernel_compartida_3(int M, float* Md, float* Nd){
    __shared__ float Nds[DIMBLOCKX];
    float Pvalue = 0;
    int columna = blockIdx.x;
    int pasos = M/ blockDim.x;
    int posIni = columna * M + threadIdx.x * pasos;
    for (int k = 0; k < pasos; ++k) {
    Pvalue = Pvalue + Md[posIni + k];
    }
    Nds[threadIdx.x] = Pvalue;
    __syncthreads();
    if (threadIdx.x == 0){
      for (int i = 1; i < blockDim.x; ++i) {
      Nds[0] = Nds[0]+Nds[i];
      }
      Nd[blockIdx.x] = Nds[0];
    }
}


// Suma por columnas de una matriz 
__global__ void SumaColMatrizKernel_4 (int M,int N,float*Md,float* Nd){
    // Pvalue es usado para el valor intermedio
    float Pvalue = 0;
    int columna = blockIdx.y*(N/gridDim.x)+threadIdx.x;
    int pasos = M/blockDim.x ;
    int posIni = columna * M + threadIdx.x * pasos;
    for (int k = 0; k < pasos; ++k) {
    Pvalue = Pvalue + Md[posIni + k];
    }
    atomicAdd(&(Nd[columna*gridDim.x+blockIdx.x]), Pvalue);
}

#define DIMBLOCKX 32
// Suma por columnas de una matriz 
__global__ void SumaColMatrizKernel_compartida_4 (int M,int N,float*Md,float* Nd){
    // Pvalue es usado para el valor intermedio
    __shared__ float Nds[DIMBLOCKX];
    float Pvalue = 0;
    int columna = blockIdx.y*(N/gridDim.x)+threadIdx.x;
    int pasos = M/blockDim.x ;
    int posIni = columna * M + threadIdx.x * pasos;
    for (int k = 0; k < pasos; ++k) {
      Pvalue = Pvalue + Md[posIni + k];
    }
    Nds[threadIdx.x] = Pvalue;
    __syncthreads();
    if (threadIdx.x == 0 ){
      for (int i = 1; i < blockDim.x; ++i) {
      Nds[0] = Nds[0]+Nds[i];
      }
      
      Nd[columna*gridDim.x+blockIdx.x] = Nds[0];
      
    }
}

void SumaColMatriz(int M, int N, float * Mh, float*  Nh){
    int size = M * N * sizeof(float), size2 = N*sizeof(float);
    float* Md, *Nd;
    // Allocate en device
    cudaMalloc(&Md, size);
    cudaMalloc(&Nd, size2);
  
    // Inicializo matrices en el device
    cudaMemcpy(Md, Mh, size, cudaMemcpyHostToDevice);
    cudaMemset(Nd,0, size2);

    /*
    ////Caso numero2
    int bloques = N / 128; // se asume N múltiplo de 128
    SumaColMatrizKernel_2<<<bloques, 128>>>( M, Md, Nd);
    // configuración de la ejecución
    dim3 tamGrid(bloques, 1); //Grid dimensión
    dim3 tamBlock(128, 1, 1); //Block dimensión
    // lanzamiento del kernel
    SumaColMatrizKernel_2<<<tamGrid, tamBlock>>>( M, Md, Nd);
    */
    
    

    /*
    ///caso numero 3
    int chunk=32;
    dim3 tamGrid(N, 1);
    dim3 tamBlock(M/chunk,1,1);
    SumaColMatrizKernel_3<<<tamGrid, tamBlock>>>( M, Md, Nd);*/
    /*
    int chunk = 32; // Se asume M múltiplo de 32
    SumaColMatrizKernel_3<<<N, M/chunk>>>(M, Md, Nd);*/

    /*
    // configuración de la ejecución
    int chunk = 32;
    dim3 tamGrid(N, 1); //Grid dimensión
    dim3 tamBlock(M/chunk,1,1); //Block dimensión
    // lanzamiento del kernel
    SumaColMatrizKernel_compartida_3<<<tamGrid, tamBlock>>>(M, Md, Nd);*/


    cout<<"Con cuatro pedasos x2"<<endl;
    ///caso numero 4
    int chunk=32;
    dim3 tamGrid(N/chunk, 1);
    dim3 tamBlock(M/chunk,chunk,1);
    SumaColMatrizKernel_compartida_4<<<tamGrid, tamBlock>>>(M,N, Md, Nd);

    /*
    // configuración de la ejecución
    int chunk=32;
    dim3 tamGrid(N/chunk, 1);
    dim3 tamBlock(M/chunk,chunk,1);
    SumaColMatrizKernel_compartida_4<<<tamGrid, tamBlock>>>(M,N, Md, Nd);*/
    

    
    /*///caso numero 4
    int chunk=32;
    dim3 tamGrid(N/chunk, 1);
    
    dim3 tamBlock(M/chunk,chunk,1);
    SumaColMatrizKernel_compartida_4<<<tamGrid, tamBlock>>>(M,N, Md, Nd);*/

    cudaMemcpy(Nh, Nd, size2, cudaMemcpyDeviceToHost);
    for (int i=0; i<N; i++)
      std::cout<<Nh[i]<<" ";

    // Free matrices en device
    cudaFree(Md); cudaFree(Nd);
}

int main(){
    int M =1024;
    int N =512;
    float *Mh = new float[M*N];//matriz host
    float *Nh = new float[N];//matriz host
    for (int i=0; i<M; i++)
      for (int j=0; j<N; j++)
        Mh[i*N+j] = 1;

    SumaColMatriz(M,N,Mh,Nh);
    cout<<"Erika animo tú puedes de verdad"<<endl;

}

Con cuatro pedasos x2
1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 1024 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 